In [3]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import faiss
from sentence_transformers import util

In [4]:
# Загрузка модели и токенизатора BERT
model_name = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [5]:
books = pd.read_csv('/Users/valeriaalesnikova/Desktop/bootcamp/smart_book_search/data/lemmatized_data.csv')
books

,Unnamed: 0,page_url,image_url,author,title,annotation,cleaned_text,lemmatized_text
0,0,https://www.biblio-globus.ru//product/10957168,https://static1.bgshop.ru/imagehandler.ashx?fi...,Маринина А.,Ричард Третий и Генрих Восьмой глазами Шекспира,Расследование жизни и деяний двух самых опороч...,расследование жизни деяний самых опороченных в...,расследование жизнь деяние самый опорочить воз...
1,1,https://www.biblio-globus.ru//product/10997075,https://static1.bgshop.ru/imagehandler.ashx?fi...,Михалкова Е.И.,Знак истинного пути,Сыщики Макар Илюшин и Сергей Бабкин расследуют...,сыщики макар илюшин сергей бабкин расследуют п...,сыщик макар илюшин сергей бабкин расследовать ...
2,2,https://www.biblio-globus.ru//product/10941892,https://static1.bgshop.ru/imagehandler.ashx?fi...,Маринина А.,Генрих Шестой глазами Шекспира,Генрих Шестой – самая трагическая фигура на ан...,генрих самая трагическая фигура английском пре...,генрих самый трагический фигура английский пре...
3,3,https://www.biblio-globus.ru//product/10978116,https://static1.bgshop.ru/imagehandler.ashx?fi...,Маринина А.,"Иоанн Безземельный, Эдуард Третий и Ричард Вто...",Истории трех знаменитых королей под одной обло...,истории трех знаменитых королей обложкой посво...,история три знаменитый король обложка посвоему...
4,4,https://www.biblio-globus.ru//product/10845007,https://static1.bgshop.ru/extraimagehandler.as...,Маринина А.,Шпаргалка для ленивых любителей истории. Корол...,Первая книга в необычном для Александры Марини...,первая необычном александры марининой жанре по...,первый необычный александра маринин жанр попул...
...,...,...,...,...,...,...,...,...
3686,3940,http://loveread.ec/view_global.php?id=16434,http://loveread.ec/img/photo_books/16434.jpg,Елена Чалова,Книга Бизнес-план для аспирантки,Ирма из повести «Бизнес-план для аспирантки» и...,ирма повести бизнесплан аспирантки помышляла б...,ирма повесть бизнесплан аспирантка помышлять б...
3687,3941,http://loveread.ec/view_global.php?id=16445,http://loveread.ec/img/photo_books/16445.jpg,Виталий Батюк,Книга Глаза Вселенной видят все…,Основная идея романа – это давление обстоятель...,основная идея романа давление обстоятельств жи...,основный идея роман давление обстоятельство жи...
3688,3942,http://loveread.ec/view_global.php?id=16452,http://loveread.ec/img/photo_books/16452.jpg,Лаура Дэниелз,Книга Для единственного…,Небольшой горный пансионат «Игл нест» в турист...,небольшой горный пансионат игл нест туристичес...,небольшой горный пансионат игла несть туристич...
3689,3943,http://loveread.ec/view_global.php?id=16461,http://loveread.ec/img/photo_books/16461.jpg,Лаура Дэниелз,Книга Ключ к сердцу,"Пола привыкла думать, что все разговоры о любв...",пола привыкла разговоры любви духовном физичес...,пол привыкнуть разговор любовь духовный физиче...


In [6]:
# Максимальная длина текста
MAX_LEN = 300

In [7]:
# Функция для встраивания текста с использованием BERT
def embed_bert_cls(text, model=model, tokenizer=tokenizer):
    t = tokenizer(text,
                  padding=True,
                  truncation=True,
                  return_tensors='pt',
                  max_length=MAX_LEN)
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().squeeze()

In [8]:
# Функция для обработки текста, чтобы все значения были строками
def preprocess_text(text):
    if isinstance(text, str):
        return text
    elif pd.isna(text):
        return ""
    else:
        return str(text)

# Применение функции к колонке 'annotation'
books['lemmatized_text'] = books['lemmatized_text'].apply(preprocess_text)

# Вычисление векторов для всех текстов в DataFrame
embeddings = []
for text in books['lemmatized_text']:  # Предполагаем, что тексты находятся в колонке 'annotation'
    embedding = embed_bert_cls(text)
    embeddings.append(embedding.numpy())

# Преобразование списка векторов в массив numpy
embeddings_array = np.stack(embeddings)

# Сохранение векторов в текстовый файл
np.savetxt('embeddings.txt', embeddings_array)

In [19]:
# Загрузка предварительно вычисленных векторов
embeddings = np.loadtxt('embeddings.txt')
embeddings_tensor = [torch.tensor(embedding) for embedding in embeddings]

# Создание индекса Faiss
embeddings_matrix = np.stack(embeddings)
index = faiss.IndexFlatIP(embeddings_matrix.shape[1])
index.add(embeddings_matrix)

# Текст запроса
text = 'Между сыщиком и преступником начинается жесткая игра'

# Встраивание запроса и поиск ближайших векторов с использованием Faiss
query_embedding = embed_bert_cls(text)
query_embedding = query_embedding.numpy().astype('float32')
k, indices = index.search(np.expand_dims(query_embedding, axis=0), 10)

# Вывод результатов
for i in indices[0]:
    print(books['title'][i])


 выход: участие в смертельно опасном реалити-шоу\
Блейз
 чтобы разгадать тайну\
Венчание со страхом
Тайна магазина игрушек
Фартовый город
Помеченный смертью
Ловцы черных душ. Дело глазника
Убийство на улице Роз
Апология здравого смысла
